In [77]:
import requests
import tqdm
import h5py

import os

import matplotlib.pyplot as plt

In [6]:
baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"f96eabbae6f336408a63f875f8cce22f"}

In [14]:
def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically
    return r

def get_save(path, fname, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

#     if r.headers['content-type'] == 'application/json':
#         return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
#         filename = r.headers['content-disposition'].split("filename=")[1]
        with open(fname, 'wb') as f:
            f.write(r.content)
        return # return the filename string

    return r

In [7]:
sim = get(baseUrl + 'TNG50-1/')

In [62]:
sim.keys()

dict_keys(['name', 'description', 'name_alt', 'boxsize', 'z_start', 'z_final', 'cosmology', 'omega_0', 'omega_L', 'omega_B', 'hubble', 'physics_model', 'has_cooling', 'has_starformation', 'has_winds', 'has_blackholes', 'mass_gas', 'mass_dm', 'softening_dm_comoving', 'softening_stars_comoving', 'softening_blackholes_comoving', 'softening_gas_comoving', 'softening_dm_max_phys', 'softening_stars_max_phys', 'softening_blackholes_max_phys', 'softening_gas_max_phys', 'softening_gas_factor', 'softening_gas_comoving_min', 'num_dm', 'num_tr_mc', 'num_tr_vel', 'longids', 'is_uniform', 'is_zoom', 'is_subbox', 'num_files_snapshot', 'num_files_groupcat', 'num_files_rockstar', 'num_files_lhalotree', 'num_files_sublink', 'num_files_ctrees', 'filesize_lhalotree', 'filesize_sublink', 'filesize_ctrees', 'filesize_ics', 'filesize_simulation', 'has_fof', 'has_subfind', 'has_rockstar', 'has_lhalotree', 'has_sublink', 'has_ctrees', 'permission_required', 'num_snapshots', 'url', 'parent_simulation', 'child_s

In [13]:
num_snapshots = sim['num_snapshots']
print(f'num snapshots = {num_snapshots}')

num_files_snapshot = sim['num_files_snapshot']
print(f'num files snapshots = {num_files_snapshot}')

num snapshots = 100
num files snapshots = 680


# Find the galaxy clusters at z = 0

Find the galaxy clusters ($M_{200}$ $\geq$ 1e13)

In [61]:
sim['files']

{'lhalotree': 'http://www.tng-project.org/api/TNG50-1/files/lhalotree/',
 'sublink': 'http://www.tng-project.org/api/TNG50-1/files/sublink/',
 'offsets': 'http://www.tng-project.org/api/TNG50-1/files/offsets/',
 'ics': 'http://www.tng-project.org/api/TNG50-1/files/ics.hdf5',
 'simulation': 'http://www.tng-project.org/api/TNG50-1/files/simulation.hdf5',
 'stellar_circs': 'http://www.tng-project.org/api/TNG50-1/files/stellar_circs.hdf5',
 'subhalo_matching_to_dark': 'http://www.tng-project.org/api/TNG50-1/files/subhalo_matching_to_dark.hdf5',
 'blackhole_mergers': 'http://www.tng-project.org/api/TNG50-1/files/blackhole_mergers.hdf5',
 'blackhole_details': 'http://www.tng-project.org/api/TNG50-1/files/blackhole_details.hdf5',
 'stellar_assembly': 'http://www.tng-project.org/api/TNG50-1/files/stellar_assembly.hdf5',
 'subbox_subhalo_list_0': 'http://www.tng-project.org/api/TNG50-1/files/subbox_subhalo_list.0.hdf5',
 'subbox_subhalo_list_1': 'http://www.tng-project.org/api/TNG50-1/files/sub

In [36]:
url = 'http://www.tng-project.org/api/TNG50-1/files/groupcat-99/'
query_mass = '?Group=Group_M_TopHat200'

get_save(url + query_mass, './initial/Group_M_TopHat200')
print('initial Group_M_TopHat200 saved')

query_firstsub = '?Group=GroupFirstSub'
get_save(url + query_firstsub, './initial/GroupFirstSub')
print('initial GroupFirstSub saved')


query_nsub = '?Group=GroupNsubs'
get_save(url + query_nsub, './initial/GroupNsubs')
print('initial GroupNsubs saved')

initial Group_M_TopHat200 saved
initial GroupFirstSub saved
initial GroupNsubs saved


In [37]:
with h5py.File('./initial/Group_M_TopHat200', 'r') as f:
    M200 = f['Group']['Group_M_TopHat200'][:] * 1e10 / 0.704

In [39]:
cl_index = np.where(M200 >= 1e13)[0]

In [47]:
with h5py.File('./initial/GroupFirstSub', 'r') as f:
    firstsub = f['Group']['GroupFirstSub'][:][cl_index]
    
with h5py.File('./initial/GroupNsubs', 'r') as f:
    nsub = f['Group']['GroupNsubs'][:][cl_index]

In [48]:
firstsub

array([     0,  63864,  96762, 117250, 143880, 167392, 184931, 198182,
       208811, 220595, 229933, 242788, 253861, 264883, 275545, 282779,
       289385, 294866, 300903, 307485, 313692, 319730, 324123, 329508,
       333424, 338446], dtype=int32)

In [49]:
nsub

array([63864, 32898, 20488, 26630, 23512, 17539, 13251, 10629, 11784,
        9338, 12855, 11073, 11022, 10662,  7234,  6606,  5481,  6037,
        6582,  6207,  6038,  4393,  5385,  3916,  5022,  4000],
      dtype=int32)

In [58]:
if not os.path.exists('./snaps/'):
    
    os.mkdir('./snaps/')
    
for i in tqdm.trange(100):
    
    path_name = f'./snaps/snap{i}'
    
    if not os.path.exists(path_name):
        os.mkdir(path_name)

100%|██████████| 100/100 [00:00<00:00, 1566.43it/s]


In [60]:
for i in tqdm.trange(100):
    
    url = f'http://www.tng-project.org/api/TNG50-1/files/groupcat-{i}/'
    path_name = f'./snaps/snap{i}/'
    
    query_mass = '?Group=Group_M_TopHat200'
    get_save(url + query_mass, path_name + 'Group_M_TopHat200')

    query_firstsub = '?Group=GroupFirstSub'
    get_save(url + query_firstsub, path_name + 'GroupFirstSub')

    query_nsub = '?Group=GroupNsubs'
    get_save(url + query_nsub, path_name + 'GroupNsubs')


100%|██████████| 100/100 [3:42:23<00:00, 133.44s/it] 


In [61]:
cl_index 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25])

# Find the progenitor cluster

In [176]:
range(firstsub[0], nsub[0]) # subhalos in first FOF group at z = 0

range(0, 63864)

In [94]:
url = f'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/' + str(0)

'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/0'

In [95]:
sub = get(url)

In [123]:
sub.keys()

dict_keys(['snap', 'id', 'bhmdot', 'cm_x', 'cm_y', 'cm_z', 'gasmetallicity', 'gasmetallicityhalfrad', 'gasmetallicitymaxrad', 'gasmetallicitysfr', 'gasmetallicitysfrweighted', 'pos_x', 'pos_y', 'pos_z', 'halfmassrad', 'halfmassrad_gas', 'halfmassrad_dm', 'halfmassrad_stars', 'halfmassrad_bhs', 'len', 'len_gas', 'len_dm', 'len_stars', 'len_bhs', 'mass', 'mass_gas', 'mass_dm', 'mass_stars', 'mass_bhs', 'massinhalfrad', 'massinhalfrad_gas', 'massinhalfrad_dm', 'massinhalfrad_stars', 'massinhalfrad_bhs', 'massinmaxrad', 'massinmaxrad_gas', 'massinmaxrad_dm', 'massinmaxrad_stars', 'massinmaxrad_bhs', 'massinrad', 'massinrad_gas', 'massinrad_dm', 'massinrad_stars', 'massinrad_bhs', 'sfr', 'sfrinhalfrad', 'sfrinmaxrad', 'sfrinrad', 'spin_x', 'spin_y', 'spin_z', 'starmetallicity', 'starmetallicityhalfrad', 'starmetallicitymaxrad', 'stellarphotometrics_u', 'stellarphotometrics_b', 'stellarphotometrics_v', 'stellarphotometrics_k', 'stellarphotometrics_g', 'stellarphotometrics_r', 'stellarphotome

**sublink_simple**

retrieve a simple representation of the SubLink tree, the Main Progenitor Branch (in 'Main') and a list of past mergers (in 'Mergers') only. In both cases a snapshot number and subhalo ID pair is given.

In [121]:
sub['trees']

{'sublink': 'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/0/sublink/full.hdf5',
 'sublink_mpb': 'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/0/sublink/mpb.hdf5',
 'sublink_mdb': 'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/0/sublink/mdb.hdf5',
 'sublink_simple': 'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/0/sublink/simple.json',
 'lhalotree': 'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/0/lhalotree/full.hdf5',
 'lhalotree_mpb': 'http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/0/lhalotree/mpb.hdf5'}

In [159]:
submpb = get_save(sub['trees']['sublink_mpb'], 'subplink_mpb')

In [165]:
submpb = h5py.File('subplink_mpb')

/home/ehko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [167]:
submpb.keys()

<KeysViewHDF5 ['DescendantID', 'FirstProgenitorID', 'FirstSubhaloInFOFGroupID', 'GroupBHMass', 'GroupBHMdot', 'GroupCM', 'GroupFirstSub', 'GroupGasMetalFractions', 'GroupGasMetallicity', 'GroupLen', 'GroupLenType', 'GroupMass', 'GroupMassType', 'GroupNsubs', 'GroupPos', 'GroupSFR', 'GroupStarMetalFractions', 'GroupStarMetallicity', 'GroupVel', 'GroupWindMass', 'Group_M_Crit200', 'Group_M_Crit500', 'Group_M_Mean200', 'Group_M_TopHat200', 'Group_R_Crit200', 'Group_R_Crit500', 'Group_R_Mean200', 'Group_R_TopHat200', 'LastProgenitorID', 'MainLeafProgenitorID', 'Mass', 'MassHistory', 'NextProgenitorID', 'NextSubhaloInFOFGroupID', 'NumParticles', 'RootDescendantID', 'SnapNum', 'SubfindID', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloCM', 'SubhaloGasMetalFractions', 'SubhaloGasMetalFractionsHalfRad', 'SubhaloGasMetalFractionsMaxRad', 'SubhaloGasMetalFractionsSfr', 'SubhaloGasMetalFractionsSfrWeighted', 'SubhaloGasMetallicity', 'SubhaloGasMetallicityHalfRad', 'SubhaloGasMetallicityMaxRad', 'Subh

In [187]:
submpb['SnapNum'][:]

array([99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83,
       82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66,
       65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49,
       48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32,
       31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
       14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0],
      dtype=int16)

In [183]:
submpb['TreeID'][:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [181]:
submpb['SubhaloID'][:]

array([90497240, 90497241, 90497242, 90497243, 90497244, 90497245,
       90497246, 90497247, 90497248, 90497249, 90497250, 90497251,
       90497252, 90497253, 90497254, 90497255, 90497256, 90497257,
       90497258, 90497259, 90497260, 90497261, 90497262, 90497263,
       90497264, 90497265, 90497266, 90497267, 90497268, 90497269,
       90497270, 90497271, 90497272, 90497273, 90497274, 90497275,
       90497276, 90497277, 90497278, 90497279, 90497280, 90497281,
       90497282, 90497283, 90497284, 90497285, 90497286, 90497287,
       90497288, 90497289, 90497290, 90497291, 90497292, 90497293,
       90497294, 90497295, 90497296, 90497297, 90497298, 90497299,
       90497300, 90497301, 90497302, 90497303, 90497304, 90497305,
       90497306, 90497307, 90497308, 90497309, 90497310, 90497311,
       90497312, 90497313, 90497314, 90497315, 90497316, 90497317,
       90497318, 90497319, 90497320, 90497321, 90497322, 90497323,
       90497324, 90497325, 90497326, 90497327, 90497328, 90497

In [179]:
submpb['FirstProgenitorID'][:]

array([90497241, 90497242, 90497243, 90497244, 90497245, 90497246,
       90497247, 90497248, 90497249, 90497250, 90497251, 90497252,
       90497253, 90497254, 90497255, 90497256, 90497257, 90497258,
       90497259, 90497260, 90497261, 90497262, 90497263, 90497264,
       90497265, 90497266, 90497267, 90497268, 90497269, 90497270,
       90497271, 90497272, 90497273, 90497274, 90497275, 90497276,
       90497277, 90497278, 90497279, 90497280, 90497281, 90497282,
       90497283, 90497284, 90497285, 90497286, 90497287, 90497288,
       90497289, 90497290, 90497291, 90497292, 90497293, 90497294,
       90497295, 90497296, 90497297, 90497298, 90497299, 90497300,
       90497301, 90497302, 90497303, 90497304, 90497305, 90497306,
       90497307, 90497308, 90497309, 90497310, 90497311, 90497312,
       90497313, 90497314, 90497315, 90497316, 90497317, 90497318,
       90497319, 90497320, 90497321, 90497322, 90497323, 90497324,
       90497325, 90497326, 90497327, 90497328, 90497329, 90497

In [171]:
submpb['FirstSubhaloInFOFGroupID'][:]

array([90497240, 90497241, 90497242, 90497243, 90497244, 90497245,
       90497246, 90497247, 90497248, 90497249, 90497250, 90497251,
       90497252, 90497253, 90497254, 90497255, 90497256, 90497257,
       90497258, 90497259, 90497260, 90497261, 90497262, 90497263,
       90497264, 90497265, 90497266, 90497267, 90497268, 90497269,
       90497270, 90497271, 90497272, 90497273, 90497274, 90497275,
       90497276, 90497277, 90497278, 90497279, 90497280, 90497281,
       90497282, 90497283, 90497284, 90497285, 90497286, 90497287,
       90497288, 90497289, 90497290, 90497291, 90497292, 90497293,
       90497294, 90497295, 90497296, 90497297, 90497298, 90497299,
       90497300, 90497301, 90497302, 90497303, 90497304, 90497305,
       90497306, 90497307, 90497308, 90497309, 90497310, 90497311,
       90497312, 90497313, 90497314, 90497315, 90497316, 90497317,
       90497318, 90497319, 90497320, 90497321, 90497322, 90497323,
       90497324, 90497325, 90497326, 90497327, 90497328, 90497

In [145]:
mpb = get_save(sub['trees']['lhalotree_mpb'], 'lhalotree_mpb')

In [146]:
mpb = h5py.File('lhalotree_mpb')

/home/ehko/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  """Entry point for launching an IPython kernel.


In [190]:
mpb.keys()

<KeysViewHDF5 ['Descendant', 'FileNr', 'FirstHaloInFOFGroup', 'FirstProgenitor', 'GroupCM', 'GroupMassType', 'Group_M_Crit200', 'Group_M_Crit500', 'Group_M_Mean200', 'Group_M_TopHat200', 'Group_R_Crit200', 'Group_R_Crit500', 'Group_R_Mean200', 'Group_R_TopHat200', 'NextHaloInFOFGroup', 'NextProgenitor', 'SnapNum', 'SubhaloBHMass', 'SubhaloBHMdot', 'SubhaloGasMetalFractions', 'SubhaloGasMetalFractionsSfr', 'SubhaloGasMetallicity', 'SubhaloGasMetallicitySfr', 'SubhaloGrNr', 'SubhaloHalfmassRad', 'SubhaloHalfmassRadType', 'SubhaloIDMostBound', 'SubhaloLen', 'SubhaloLenType', 'SubhaloMassInRadType', 'SubhaloMassType', 'SubhaloNumber', 'SubhaloOffsetType', 'SubhaloPos', 'SubhaloSFR', 'SubhaloSFRinRad', 'SubhaloSpin', 'SubhaloStarMetalFractions', 'SubhaloStarMetallicity', 'SubhaloStellarPhotometrics', 'SubhaloVMax', 'SubhaloVel', 'SubhaloVelDisp']>

In [189]:
mpb['SnapNum'][:]

array([99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83,
       82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66,
       65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49,
       48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32,
       31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
       14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0],
      dtype=int32)

In [158]:
mpb['SubhaloNumber'][:]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0, 15511,
       15404, 15624, 15693, 15268, 15472, 15216, 15220, 15195,  9247,
        8857,  8266, 15343,  8068, 14355, 13700, 13673, 13021,  9440,
       10323,  8987,  3769,  3258,  5996,  4230,  3857,  3618,  3738,
        2918,  3119,  1836,  4458,  4301,  4308,  3400,  2314,  2120,
        2014,  1470,  1393,  1317,  1329,  3971,  6134,  4334,  3853,
        1171], dtype=int32)

In [126]:
temp = get(sub['trees']['sublink_simple']) #Main, Merger

In [140]:
temp2 = np.array(temp['Mergers'])

In [175]:
temp2[(temp2[:, 0] == 98) & (temp2[:, 1] == 0)]

array([], shape=(0, 2), dtype=int64)

In [ ]:
for i in range(100)

    # request the full subhalo details of the descendant by following the sublink URL
    sub = get(sub['related']['sublink_projenitor'])

In [ ]:
#find halos whose masses are greater than 14.
# link the halos in the direction of redshift
# check gas, star-forming galxy distribution

#reference: IllustriTNG-50 page